In [7]:
import pytimetk as tk
import pandas as pd
import pickle

## The purpose of this notebook is to 
1. Decompose the momentum calculation to better understand how it works
2. See if we can speed up the momentum calculations by using pytimetk


Initial results appears to reduce calculation times from 11 minutes to 1 second

The momentum figures don't yet match the results returned by 02_momentum.ipynb as I don't know how to exactly translate the logic between pandas and pytimetk



## Original Logic (Reference Only)

In [58]:
def momentum(close):
    returns = close.pct_change()[-126:]
    return(
        (close[-21] - close[-252]) / close[-252] 
        - (close[-1] - close[-21]) / close[-21]
    ) / np.std(returns)

The prices.pkl data comes from 02_momentum.ipynb file and written before the any calcs have been done on the dataframe. 

In [11]:
with open('prices.pkl', 'rb') as file:
    df = pickle.load(file)

In [55]:
def momentum_tk(df):
    df = (
        df
            .reset_index()
            .groupby('symbol')
            .augment_lags(
                date_column='date',
                value_column='close',
                lags=[1,21,126,252],
                engine='polars'
            )
    )
    df['return126'] = df.groupby('symbol')['close'].pct_change(126) 
    df = (
        df
            .groupby('symbol')
            .augment_rolling(
                date_column='date',
                value_column='return126',
                window=[126],
                window_func = ['std'],
                engine='polars'
            )
    )
    df['momentum'] = (
            (df['close_lag_21'] - df['close_lag_252']) / df['close_lag_252'] 
            - (df['close_lag_1'] - df['close_lag_21']) / df['close_lag_21']
        ) / df['return126_rolling_std_win_126']
    
    return df
result = momentum_tk(df)
result

,symbol,date,open,high,low,close,volume,close_lag_1,close_lag_21,close_lag_126,close_lag_252,return126,return126_rolling_std_win_126,momentum
0,ZTS,2015-01-02,40.726363,40.951269,40.360895,40.585800,1784200,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ZTS,2015-01-05,40.529588,40.885687,40.267201,40.342167,3112100,40.585800,NaN,NaN,NaN,NaN,NaN,NaN
2,ZTS,2015-01-06,40.435866,40.632657,39.639329,39.948574,3977200,40.342167,NaN,NaN,NaN,NaN,NaN,NaN
3,ZTS,2015-01-07,40.295301,40.820078,40.276558,40.773220,2481800,39.948574,NaN,NaN,NaN,NaN,NaN,NaN
4,ZTS,2015-01-08,41.935232,42.047683,41.232407,41.401085,3121300,40.773220,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
424185,AAOI,2024-01-19,16.610001,16.798000,15.020000,15.810000,3511500,16.620001,22.500000,8.12,2.37,0.947044,1.438291,6.087091
424186,AAOI,2024-01-22,16.100000,17.190001,15.890000,17.100000,2396400,15.810000,23.010000,7.91,2.29,1.161821,1.450823,6.452162
424187,AAOI,2024-01-23,17.180000,18.629999,17.180000,18.129999,2990600,17.100000,20.620001,8.52,2.21,1.127934,1.465563,5.800519
424188,AAOI,2024-01-24,18.500000,18.799999,16.549999,16.590000,1939900,18.129999,20.740000,8.43,2.24,0.967971,1.481814,5.658451


In [57]:
result.groupby('symbol').tail(1).sort_values('momentum', ascending=False)

,symbol,date,open,high,low,close,volume,close_lag_1,close_lag_21,close_lag_126,close_lag_252,return126,return126_rolling_std_win_126,momentum
62895,UBER,2024-01-25,64.279999,66.029999,63.709999,65.565002,13503906,63.759998,61.709999,47.310001,29.930000,0.385859,0.088871,11.573929
156141,PANW,2024-01-25,344.459991,343.739990,337.500000,338.019989,2090935,340.239990,298.209991,245.009995,151.139999,0.379617,0.090889,9.155478
124333,RCL,2024-01-25,126.589996,126.959999,125.809998,126.714996,799183,125.489998,128.210007,100.879997,63.980000,0.256096,0.122402,8.375077
144736,PHM,2024-01-25,103.129997,104.375000,102.910004,103.989998,920356,102.360001,102.430000,83.964188,51.156456,0.238504,0.120682,8.310837
380822,AVGO,2024-01-25,1281.000000,1280.439941,1240.780029,1240.900024,1464937,1253.869995,1121.979980,884.186584,572.011353,0.403437,0.102735,8.214493
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115899,RTX,2024-01-25,90.250000,91.105003,89.580002,90.654999,4090594,89.489998,83.230003,85.074860,96.925491,0.065591,0.064017,-3.382096
310884,DG,2024-01-25,132.089996,133.111099,128.759995,132.770004,1072234,130.669998,131.635437,163.252991,230.360458,-0.186722,0.122471,-3.439458
370662,BOIL,2024-01-25,26.730000,26.889999,24.090000,24.100000,9471737,26.780001,28.580000,64.570000,194.399994,-0.626762,0.218419,-3.616915
293471,EL,2024-01-25,127.500000,129.490005,125.540001,129.449997,1640237,126.160004,144.429993,176.403442,272.890015,-0.266171,0.067644,-5.089050
